---

# **Notebook 18: Analyse des Valeurs Aberrantes et Nettoyage des Données pour la Modélisation**

---

### **1. Importation des Bibliothèques**

Nous importons les bibliothèques nécessaires pour la manipulation des données et la création de graphiques.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

---

### **2. Définition des Répertoires d'Entrée et de Sortie**

Nous définissons les répertoires contenant les fichiers de données à traiter et celui où les fichiers nettoyés et les graphiques seront sauvegardés.

In [ ]:
# Répertoires d'entrée et de sortie
input_directory = 'note-books/phase_1_data/17_cleaned_data'
output_directory = 'note-books/phase_1_data/18_cleaned_for_modeling'
output_graphs_directory = 'note-books/phase_1_data/18_graphs'
os.makedirs(output_directory, exist_ok=True)
os.makedirs(output_graphs_directory, exist_ok=True)

In [ ]:
---

### **3. Fonction de Détection et de Suppression des Valeurs Extrêmes**

Cette fonction identifie les valeurs aberrantes en fonction de l'écart interquartile (IQR) et les remplace par `NaN`.

In [ ]:
def nettoyer_outliers(df, column):
    """
    Remplace les valeurs extrêmes dans une colonne donnée en utilisant la méthode IQR.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), np.nan, df[column])

---

### **4. Chargement, Visualisation et Nettoyage des Fichiers**

Pour chaque fichier dans le dossier d’entrée :
1. Charger les données et filtrer les colonnes numériques.
2. Générer un graphique de boîte pour la distribution des données avant nettoyage.
3. Appliquer la fonction de nettoyage des valeurs extrêmes.
4. Générer un graphique de boîte après nettoyage.
5. Enregistrer le fichier nettoyé.

In [ ]:
# Liste pour stocker les statistiques des valeurs min et max
outliers_data = []

for file_name in os.listdir(input_directory):
    file_path = os.path.join(input_directory, file_name)
    
    # Vérifier si le fichier est vide
    if os.path.getsize(file_path) == 0:
        print(f"{file_name} est vide et sera ignoré.")
        continue
    
    # Charger le fichier
    try:
        df = pd.read_csv(file_path)
        if df.empty:
            print(f"{file_name} ne contient aucune donnée. Ignoré.")
            continue
    except pd.errors.EmptyDataError:
        print(f"{file_name} est vide et sera ignoré.")
        continue

    # Sélectionner uniquement les colonnes numériques
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Calculer les valeurs min et max pour les statistiques
    stats = numeric_df.describe().T[['min', 'max']]
    stats['column'] = stats.index
    stats['file'] = file_name
    outliers_data.append(stats)
    
    # Générer le graphique de boîte avant nettoyage
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=numeric_df, palette="Set3")
    plt.title(f'Distribution avant nettoyage - {file_name}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    pre_clean_graph_path = os.path.join(output_graphs_directory, f"{file_name}_before_cleaning.png")
    plt.savefig(pre_clean_graph_path)
    plt.close()
    print(f"Graphique avant nettoyage sauvegardé : {pre_clean_graph_path}")
    
    # Appliquer le nettoyage des valeurs extrêmes
    for col in numeric_df.columns:
        nettoyer_outliers(df, col)
    
    # Générer le graphique de boîte après nettoyage
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df.select_dtypes(include=[np.number]), palette="Set2")
    plt.title(f'Distribution après nettoyage - {file_name}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    post_clean_graph_path = os.path.join(output_graphs_directory, f"{file_name}_after_cleaning.png")
    plt.savefig(post_clean_graph_path)
    plt.close()
    print(f"Graphique après nettoyage sauvegardé : {post_clean_graph_path}")
    
    # Sauvegarder le fichier nettoyé
    output_file_path = os.path.join(output_directory, file_name)
    df.to_csv(output_file_path, index=False)
    print(f"Fichier nettoyé et sauvegardé : {output_file_path}")

# Fusionner les données de statistiques et enregistrer en fichier texte
if outliers_data:
    all_outliers = pd.concat(outliers_data, ignore_index=True)
    output_stats_path = os.path.join(output_directory, 'outliers_summary.txt')
    with open(output_stats_path, 'w') as f:
        f.write("Valeurs min et max par colonne et par fichier\n\n")
        f.write(all_outliers.to_string(index=False))
    print(f"Résumé des valeurs extrêmes sauvegardé dans {output_stats_path}")
else:
    print("Aucune donnée numérique trouvée dans les fichiers traités.")

---

### **5. Explications**

- **Visualisation avant et après nettoyage** : Les graphiques permettent de visualiser les valeurs extrêmes avant et après leur traitement pour vérifier leur suppression.
- **Méthode de nettoyage des valeurs extrêmes** : La méthode IQR identifie les valeurs aberrantes, qui sont ensuite remplacées par `NaN`.
- **Organisation structurée** : Les fichiers nettoyés et les graphiques sont sauvegardés pour une analyse détaillée.

---